In [1]:
import pandas as pd
import numpy as np
import datetime
import io

In [11]:
#This script will summary all the transaction data assuming that you have the block # of the transaction, the gas amt 
#the gas price and the time stamp.
def summary(x): #save your file to a variable.  If your file is in a different folder than your script, you need to save the
    #entier path to the file.  x = varible for file name or path and file name
    gas = pd.read_csv(x) #reading contents of file into a data frame
    gas.rename(columns={"block_number": "Block_num", "gas_price": "Gas_price", "receipt_gas_used" : "Gas_amt",\
                        "block_timestamp" : "Day_Time"},inplace = True) #changing column names
                                                                                                        
    gas['Gas_price'] = gas['Gas_price']/1000000000 #putting gas in easier to read form
    gas1 = gas.groupby(['Block_num', 'Day_Time']) #grouping by block number
    total_fee = gas['Gas_price']*gas['Gas_amt'] #Total transaction fee gas price times gas amount
    gas['Total_fee'] = total_fee  #adding total fee to the dataframe
    
    #creating summaries by block.
    gas_mean = gas1.Gas_price.mean()  #gas price mean by block
    gas_median = gas1.Gas_price.median() #gas price median by block
    gasmin = gas1.Gas_price.min() #gas price min by block
    gasmax = gas1.Gas_price.max() #gas price max by block
    tmin = gas1.Total_fee.min() #gas total fee min by block
    tmax = gas1.Total_fee.max() #gas total fee max by block
    tmed = gas1.Total_fee.median() #gas total fee median by block
    tmean = gas1.Total_fee.mean() #gas total fee mean by block
    tsum = gas1.Total_fee.sum() # total transactions fees by block
    blockcount = gas1.Block_num.count() #number of transactions in the block
    gassummary = pd.DataFrame({'Gas_mean' : gas_mean, 'Gas_median': gas_median, 'Gas_min': gasmin, 'Gas_max': gasmax,
                          'Transfee_min': tmin, 'Transfee_max': tmax, 'Transfee_median': tmed, 'Transfee_mean': tmean,
                          'Transfee_Total': tsum, 'Count_perBlock': blockcount})
    gassummary.reset_index(inplace = True)
    gassummary.set_index('Block_num', inplace = True)
    return(gassummary)

    

In [13]:
march = '03_22_03_26.csv' #This is data pulled from bigquery to match the dates from the available pending transactions.
marchdata = pd.read_csv(march)

In [80]:
marchsummary = summary(march) #running script

In [81]:

marchsummary.sort_values(by = ['Day_Time'], inplace = True) #sorting block date time
marchsummary.reset_index(inplace = True)
marchsummary.set_index(['Day_Time'], inplace = True) #setting the block timestamp as index
marchsummary.index = pd.to_datetime(marchsummary.index) #changing string to datetime

In [83]:
pending = 'pending.csv' #number of pending transactions per minute from https://etherscan.io/chart/pendingtx
pendingdata = pd.read_csv(pending)    

In [84]:
timerange = marchsummary['2021-03-22 15:16:00' : '2021-03-26 15:42:00' ].copy(deep = True) #copying times needed this should
#match the time range from the pending file

In [85]:
pendingdata.rename(columns={"Date(UTC)": "Day_Time"}, inplace = True) #changing column for ease

In [87]:
pendingdata.Day_Time = pd.to_datetime(pendingdata.Day_Time) #changing pending day time value to type datetime


In [68]:
#This script matches each minute from the pending transactions file to the blocks within that minute
def pendingfunc(pendingdata, timerange):
    timerange.insert(loc = 10, column = "number_pending", value = " ") #inserting a column for pending
    n= len(pendingdata)
    for i in range(0, n): #looping through each time available
        begin = pendingdata['Day_Time'].astype(str)[i] + '+00:00' #begining time for search
        end =  pendingdata['Day_Time'].astype(str)[i]  #ending time for search
        if end.count("00") == 1: #changing 00 to 59 to account for minute
            end = end.replace("00", "59")
        else:
            end = end.replace("00", "59")
            end = end.replace("59", "00", 1)
        end = end + '+00:00'
        
        timerange[begin : end]["number_pending"].replace(" ", pendingdata['Value'][i], inplace = True) 
        #searching for all blocks during minute and adding the number of pending into column
    

    
    

In [88]:
pendingfunc(pendingdata, timerange) #FYI this takes about 3 to 5 minutes and a warning will appear about setting a 
#value to a copy of a slice of a data frame.  I tried to do a deep copy above, but I am not sure why it does not work
#there is likely a quicker way to write this function, given the time for it to run, but again, this is the best 
#I have for now.  Feel free to change the code 

C:\Users\Sara\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [214]:

marchdata.sort_values(by = ['block_timestamp'], inplace = True) 
marchdata.set_index(['block_timestamp'], inplace = True) #setting the block timestamp as index
marchdata.index = pd.to_datetime(marchdata.index)
hashes = marchdata['2021-03-22 15:16:00' : '2021-03-26 15:42:00']['hash'] #getting possible transaction #s during time 
number = len(hashes) #finding the total number of transactions
observations = np.random.choice(number, 500, replace = False) #generating 500 randomly selected index # for observations
transactionnum = [hashes[i] for i in observations] #selecting transaction numbers based on random index


In [223]:

transactionnum = pd.DataFrame(transactionnum) #saving list as a dataframe
transactionnum.rename(columns = {0 : 'hash'}, inplace = True) #changing column name from 0 to hash
transactionnum.to_csv("transaction_march.csv") #outputting data frame to file for use with scraper for confirmation times
#I used a scraper in R

5281996

In [116]:
obsdata = pd.read_csv('observations.csv') #loading observations which scraped from web.  Information included in file
# transaction hash, block, totalfee for the transaction, date, time, confirmation (how long waiting for a block)
#gas price and gas amount.

In [131]:
#merging data from observations confirmation time to pending transactions and summary information
obsdata.reset_index(inplace = True) #resetting index
obsdata.drop(columns = ['Unnamed: 0', 'index', 'date', 'time', 'totalfee'], inplace = True) #getting rid of columns duplicated by 
other file that will be merged.
obsdata.set_index(['hash'], inplace = True) #setting hash as index to select needed rows from summary file
ind = obsdata.index #saving index to a list
marchdata.set_index(['hash'], inplace = True) #setting index to hash to use for searching with ind
observations = marchdata.loc[ind] #getting a data frame that links hash # and block # of selected observations
observations.drop(columns = ['block_timestamp', 'receipt_gas_used', 'gas_price'], inplace = True) #getting rid of duplicate 
columns for merge
observations.reset_index(inplace = True) #resetting index without losing hash
observations.set_index(['block_number'], inplace = True) #setting index to block number for search on time range
ind = observations.index #saving index to a list
timerange.reset_index(inplace = True) #resetting index so as not to lose Day_Time
timerange.set_index(['Block_num'], inplace = True) #setting block number to index for search by observation index
observations = timerange.loc[ind] selecting the observations from time range
observations.reset_index(inplace = True)
obsdata.reset_index(inplace = True) #undoing hash as index
obsdata.rename(columns = {'block': 'block_number'}, inplace = True) #changing name of column for merge
observations = pd.merge(observations, obsdata, on = 'block_number' )
observations['totalfee'] = observations['gasprice']*observations['gasamt'] #adding back transaction fee for observation
observations.drop_duplicates(inplace = True) #dropping duplicates (not sure how this happened)


,block_number,Day_Time,Gas_mean,Gas_median,Gas_min,Gas_max,Transfee_min,Transfee_max,Transfee_median,Transfee_mean,Transfee_Total,number_pending,Count_perBlock,hash,confirmation,gasprice,gasamt,totalfee
0,12096861,2021-03-23 18:53:48+00:00,241.247559,247.53,194.000000,490.600000,4074000.0,1.086219e+08,8.560506e+06,1.307668e+07,2.876871e+09,162165,220,0x802f6ad21746bc17db4ace776a8b16a04aa87d3c5ad3...,30.0,244.879875,21000,5.142477e+06
1,12090438,2021-03-22 19:10:34+00:00,193.571644,177.00,175.000001,400.000000,3717000.0,7.019040e+07,3.717000e+06,7.264672e+06,2.593488e+09,177384,357,0xffe6031a6146cf34ac620a160256f8d5a54293074e97...,1227.0,177.000000,21000,3.717000e+06
2,12090438,2021-03-22 19:10:34+00:00,193.571644,177.00,175.000001,400.000000,3717000.0,7.019040e+07,3.717000e+06,7.264672e+06,2.593488e+09,177384,357,0xa9c3b2a811711a47d718201977c50b367ac401e6a370...,1221.0,177.000000,21000,3.717000e+06
5,12107514,2021-03-25 10:12:56+00:00,151.707338,145.00,124.000000,250.000000,2604000.0,8.029359e+07,6.343194e+06,9.318089e+06,1.854300e+09,184585,199,0xc30376d961400158f4dd381d1817bd11180ca003b3a9...,30.0,150.000000,21000,3.150000e+06
6,12105891,2021-03-25 04:20:29+00:00,243.765043,146.00,0.000000,7631.851164,0.0,1.657585e+08,4.067953e+06,1.077659e+07,2.166094e+09,168169,201,0x12a3bb29ca16fa4b5887ece6d4cb86ffdbc5f42950a7...,30.0,135.000000,111618,1.506843e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,12092040,2021-03-23 01:04:11+00:00,190.714814,181.00,154.000000,349.700000,3284295.0,1.957340e+08,4.743829e+06,8.787636e+06,2.372662e+09,185703,270,0xe3f0ba7c85554b72d4ec320c8c2167d83d0b8659fa00...,NaN,181.000000,21000,3.801000e+06
510,12102199,2021-03-24 14:47:58+00:00,252.972987,223.00,201.680000,2700.813590,4235280.0,2.120436e+08,7.563984e+06,1.207100e+07,3.126390e+09,160854,259,0xfdd52ee72cea34b8cdc0297a263c77d5c02179f1ebf1...,30.0,306.000000,44474,1.360904e+07
511,12112407,2021-03-26 04:20:51+00:00,123.768557,120.00,113.000000,226.250000,2373000.0,7.473257e+07,4.698510e+06,8.454249e+06,1.521765e+09,164081,180,0x6d1237ad49a9b7abe0168c02ad2a8529afba0c7392ac...,30.0,125.000000,46661,5.832625e+06
512,12100210,2021-03-24 07:18:27+00:00,137.368280,139.00,10.000000,226.000000,210000.0,9.375467e+07,3.253276e+06,6.991699e+06,1.678008e+09,165222,240,0x29bbce2ff9a5326b4a159492d30cf786dc839131e8c9...,8.0,146.000000,21000,3.066000e+06


In [133]:
#observations.to_csv("observations_2.csv") #output file